In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
import requests
from io import BytesIO
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('consump_r_daily_agg.csv')
df['Date_Time'] = pd.to_datetime(df['Date_Time'])
df.Electricity_KW_AVG = np.log(df.Electricity_KW_AVG)
df.rename(columns={'Electricity_KW_AVG':'Electricity_Log'}, inplace=True)

In [2]:
df.head(2)

,Date_Time,Weekend,Holiday,School_Day,Electricity_Log,Solar_Elevation_AVG,Cloud_Cover_Fraction_AVG,Dew_Point_AVG,Humidity_Fraction_AVG,Precipitable_Water_AVG,...,Day_of_week_2,Day_of_week_3,Day_of_week_4,Day_of_week_5,Day_of_week_6,Day_of_week_7,Season_Autumn,Season_Spring,Season_Summer,Season_Winter
0,1991-01-01,1,1,0,11.057840,-18.476732,0.950000,-12.883333,0.763750,5.208333,...,0,0,0,0,0,0,0,0,0,1
1,1991-01-02,0,0,0,11.074369,-18.411217,0.458333,-14.670833,0.679583,4.833333,...,1,0,0,0,0,0,0,0,0,1


In [3]:
time = df[['Date_Time', 'Electricity_Log']]
del df['Date_Time']
del df['Electricity_Log']
df.head()

,Weekend,Holiday,School_Day,Solar_Elevation_AVG,Cloud_Cover_Fraction_AVG,Dew_Point_AVG,Humidity_Fraction_AVG,Precipitable_Water_AVG,Temperature_AVG,Visibility_AVG,...,Day_of_week_2,Day_of_week_3,Day_of_week_4,Day_of_week_5,Day_of_week_6,Day_of_week_7,Season_Autumn,Season_Spring,Season_Summer,Season_Winter
0,1,1,0,-18.476732,0.950000,-12.883333,0.763750,5.208333,-9.441667,8.470833,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,-18.411217,0.458333,-14.670833,0.679583,4.833333,-9.570833,16.100000,...,1,0,0,0,0,0,0,0,0,1
2,0,0,0,-18.339779,0.583333,-11.320833,0.610417,5.583333,-4.670833,16.100000,...,0,1,0,0,0,0,0,0,0,1
3,0,0,0,-18.262442,0.862500,-13.825000,0.746250,5.083333,-10.016667,8.725000,...,0,0,1,0,0,0,0,0,0,1
4,0,0,0,-18.179237,0.554167,-20.070833,0.757083,3.833333,-16.716667,10.675000,...,0,0,0,1,0,0,0,0,0,1


In [4]:
scaler = MinMaxScaler()
scaler.fit(df)

exo = pd.DataFrame(scaler.transform(df))

C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
train_time = np.array(time.Electricity_Log[0:292])
test_time = np.array(time.Electricity_Log[292:])
train_exo = np.array(exo[0:292])
test_exo = np.array(exo[292:])

In [6]:
train_time.shape

(292,)

In [11]:
train_exo.shape
from sklearn.metrics import mean_squared_error, explained_variance_score

In [15]:
model = SARIMAX(train_time, exog=train_exo, order=(1, 1, 0), seasonal_order=(2, 0, 1, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('1')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

model = SARIMAX(train_time, exog=train_exo, order=(2, 1, 0), seasonal_order=(2, 0, 1, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('2')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

model = SARIMAX(train_time, exog=train_exo, order=(3, 1, 0), seasonal_order=(2, 0, 1, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('3')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

model = SARIMAX(train_time, exog=train_exo, order=(1, 1, 1), seasonal_order=(2, 0, 1, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('4')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

model = SARIMAX(train_time, exog=train_exo, order=(2, 2, 1), seasonal_order=(2, 0, 1, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('5')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

model = SARIMAX(train_time, exog=train_exo, order=(3, 1, 1), seasonal_order=(2, 0, 1, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('6')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

model = SARIMAX(train_time, exog=train_exo, order=(1, 1, 2), seasonal_order=(2, 0, 1, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('7')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

model = SARIMAX(train_time, exog=train_exo, order=(2, 1, 2), seasonal_order=(2, 0, 1, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('8')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

model = SARIMAX(train_time, exog=train_exo, order=(3, 1, 2), seasonal_order=(2, 0, 1, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('9')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

model = SARIMAX(train_time, exog=train_exo, order=(3, 1, 3), seasonal_order=(2, 0, 0, 7), trend = 'ct')
model_fit = model.fit(disp=True)
fc = model_fit.forecast(73, alpha=0.05, exog = pd.DataFrame(test_exo) )
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print('10')
print(rmse)
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)

C:\Users\sajor\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


1
7412.896449899534
-17.531394239784202


C:\Users\sajor\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


2
7639.033430917428
-18.583836371240512


C:\Users\sajor\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


...

8
8976.621863060944
-22.374900910971178


C:\Users\sajor\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


9
7455.523283617435
-17.681248749898113


C:\Users\sajor\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:949: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\sajor\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:961: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


10
7731.543376175832
-18.814826714797025


In [ ]:
#test_exo.shape

In [ ]:
#fc

In [ ]:
#test_time

In [ ]:
#np.exp(fc)

In [ ]:
#np.exp(test_time)

In [ ]:
actual = pd.DataFrame(np.exp(test_time))
actual.index = time[292:].index
forecast = pd.DataFrame(np.exp(fc))
forecast.index = actual.index

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(np.exp(train_time), label='training')
plt.plot(actual, label='actual')
plt.plot(forecast, label='forecast', alpha = 0.7)
#plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, lower_series, upper_series, 
                 #color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(actual.values, forecast.values)
rmse = np.sqrt(mse)
print(rmse)

from sklearn.metrics import explained_variance_score
r2 = explained_variance_score(actual.values, forecast.values)
print(r2)